In [9]:
import os

# dask/parallelization libraries
import coiled
import dask
from dask.distributed import Client, LocalCluster
from dask.distributed import print as dask_print
import dask.config
import distributed

import numpy as np
import rasterio
import rasterio.features
import rasterio.transform
import rasterio.windows
from osgeo import gdal
from subprocess import check_call

from numba import jit
import concurrent.futures

import boto3
import time
import math
import ctypes
import pandas as pd

<font size="6">Making cloud and local clusters</font> 

In [ ]:
# Test cluster
coiled_cluster = coiled.Cluster(
    n_workers=1,
    use_best_zone=True, 
    compute_purchase_option="spot_with_fallback",
    idle_timeout="10 minutes",
    region="us-east-1",
    name="next_gen_forest_carbon_flux_model", 
    account='jterry64', # Necessary to use the AWS environment that Justin set up in Coiled
    worker_memory = "32GiB" 
)

# Coiled cluster (cloud run)
coiled_client = coiled_cluster.get_client()
coiled_client

In [75]:
# Full cluster
# Fewer workers with somewhat more memory. Even this configuration has workers entering the red zone and running out of memory but it seems to be okay
coiled_cluster = coiled.Cluster(
    n_workers=18,
    use_best_zone=True, 
    compute_purchase_option="spot_with_fallback",
    idle_timeout="10 minutes",
    region="us-east-1",
    name="next_gen_forest_carbon_flux_model", 
    account='jterry64', # Necessary to use the AWS environment that Justin set up in Coiled
    worker_memory = "205GiB" 
)

# Coiled cluster (cloud run)
coiled_client = coiled_cluster.get_client()
coiled_client

Output()

Package - bcrypt, bcrypt==4.1.0 has no install candidate for linux-64 on conda-forge

╭────────────────────────────────────────── Package Info ──────────────────────────────────────────╮
│                                  ╷                                                               │
│   Package                        │ Note                                                          │
│ ╶────────────────────────────────┼─────────────────────────────────────────────────────────────╴ │
│   et-xmlfile                     │ https://pypi.org/pypi                                         │
│                                  ╵                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────── Not Synced with Cluster ─────────────────────────────────────╮
│            ╷                                                                         ╷           │
│   Package  │ Error                                                                   │ Risk      │
│ ╶──────────┼─────────────────────────────────────────────────────────────────────────┼─────────╴ │
│   bcrypt   │ bcrypt==4.1.0 has no install candidate for linux-64 on conda-forge      │ Warning   │
│            ╵                                                                         ╵           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

<Client: 'tls://10.0.8.142:8786' processes=9 threads=432, memory=1.62 TiB>

In [ ]:
# Coiled cluster (cloud run)
coiled_client = coiled_cluster.get_client()
coiled_client

In [ ]:
# Local single-process cluster (local run). Will run .compute() on just one process, not a whole cluster.
local_client = Client(processes=False)
local_client

In [ ]:
local_client = Client()
local_client

In [ ]:
# Local cluster with multiple workers
local_cluster = LocalCluster()  
local_client = Client(local_cluster)
local_client

<font size="6">Shutting down cloud and local clusters</font> 

In [81]:
coiled_cluster.shutdown()

In [ ]:
local_client.shutdown()

<font size="6">Analysis</font> 

<font size="4">Paths and functions</font>

In [12]:
# General paths and constants

input_uri = 's3://gfw2-data/forest_change/GLAD_Europe_height_data/202312_published/'
output_suffix = 'processed/20231227/'

def timestr():
    return time.strftime("%Y%m%d_%H_%M_%S")

In [4]:
# Returns list of all chunk boundaries within a bounding box for chunks of a given size
def get_chunk_bounds(chunk_params):

    min_x = chunk_params[0]
    min_y = chunk_params[1]
    max_x = chunk_params[2]
    max_y = chunk_params[3]
    chunk_size = chunk_params[4]
    
    x, y = (min_x, min_y)
    chunks = []

    # Polygon Size
    while y < max_y:
        while x < max_x:
            bounds = [
                x,
                y,
                x + chunk_size,
                y + chunk_size,
            ]
            chunks.append(bounds)
            x += chunk_size
        x = min_x
        y += chunk_size

    return chunks

# Returns the encompassing tile_id string in the form YYN/S_XXXE/W based on a coordinate
def xy_to_tile_id(top_left_x, top_left_y):

    lat_ceil = math.ceil(top_left_y/10.0) * 10
    lng_floor = math.floor(top_left_x/10.0) * 10
    
    lng: str = f"{str(lng_floor).zfill(3)}E" if (lng_floor >= 0) else f"{str(-lng_floor).zfill(3)}W"
    lat: str = f"{str(lat_ceil).zfill(2)}N" if (lat_ceil >= 0) else f"{str(-lat_ceil).zfill(2)}S"

    return f"{lat}_{lng}"

In [5]:
# Lazily opens tile within provided bounds (i.e. one chunk) and returns as a numpy array
# If it can't open the chunk (no data in it), it returns an array of all 0s
def get_tile_dataset_rio(uri, bounds, chunk_length):

    try:
        with rasterio.open(uri) as ds:
            window = rasterio.windows.from_bounds(*bounds, ds.transform)
            data = ds.read(1, window=window)
    except:
        data = np.zeros((chunk_length, chunk_length))

    if data.size==0:
        # dask_print("No data in chunk")
        return np.zeros((chunk_length, chunk_length))
    else:
        # dask_print("Data in chunk")
        return data

<font size="4">Model steps</font>

In [79]:
# Cuts Europe-size rasters to GLAD 10x10 tiles
def translate_to_GLAD_tile(bounds, chunk_length_deg, year):
 
    futures = {}
    layers = {}

    bounds_str = "_".join([str(round(x)) for x in bounds])
    chunk_length_pixels = int(chunk_length_deg * (40000/10))

    block_size = 400

    # Submit requests to S3 for input chunks but don't actually download them yet. This queueing of the requests before downloading them speeds up the downloading
    with concurrent.futures.ThreadPoolExecutor() as executor:
        
        tile_id = xy_to_tile_id(bounds[0], bounds[3])

        tree_height_uri = f'{input_uri}tree_height/raw/Tree_height_{year}.tif'
        tree_extent_uri = f'{input_uri}tree_extent/raw/Tree_extent_{year}.tif'
        tree_removal_uri = f'{input_uri}tree_removal/raw/Tree_removal_{year}.tif'
        
        futures[executor.submit(get_tile_dataset_rio, tree_height_uri, bounds, chunk_length_pixels)] = f"tree_height_{year}"
        futures[executor.submit(get_tile_dataset_rio, tree_extent_uri, bounds, chunk_length_pixels)] = f"tree_extent_{year}"
        futures[executor.submit(get_tile_dataset_rio, tree_removal_uri, bounds, chunk_length_pixels)] = f"tree_removal_{year}"


    # Wait for requests to come back with data from S3
    for future in concurrent.futures.as_completed(futures):
        layer = futures[future]
        layers[layer] = future.result()

    # Skips chunk if it has no forest extent in it
    if not np.any(layers[f"tree_height_{year}"]):
        dask_print(f"No data in chunk {bounds_str} for {year}. Skipping: {timestr()}")
        return 
    
    dask_print(f"Data in chunk {bounds_str} for {year}. Proceeding: {timestr()}.")

    transform = rasterio.transform.from_bounds(*bounds, width=chunk_length_pixels, height=chunk_length_pixels)

    # Output files to upload to s3
    output_dict = {
        "tree_height": [layers[f"tree_height_{year}"], f"forest_change/GLAD_Europe_height_data/202312_published/tree_height/{output_suffix}"],
        "tree_extent": [layers[f"tree_extent_{year}"], f"forest_change/GLAD_Europe_height_data/202312_published/tree_extent/{output_suffix}"],
        "tree_removal": [layers[f"tree_removal_{year}"], f"forest_change/GLAD_Europe_height_data/202312_published/tree_removal/{output_suffix}"]                          
    }

    s3_client = boto3.client("s3")

    # For every output file, saves from array to local raster, then to s3.
    # Can't save directly to s3, unfortunately, so need to save locally first.
    for key, value in output_dict.items():

        data = value[0]

        # If the originl raster doesn't go all the way to the edges of the chunk, the numpy array must be padded to achieve the desired dimensions
        # Define the target dimensions
        target_shape = (chunk_length_pixels, chunk_length_pixels)
        

            
        # Pads the rows to 40,000 rows
        # Special case for northeastern-most. Without this, the padding would be rows below the data and the data would be erronesously at the top of the tile.
        # The order of the padding matters, as far as I can tell.
        if tile_id == "80N_030E":
            dask_print(f"  Doing special 80N padding for {key} {bounds_str} in {tile_id} for {year}: {timestr()}") 

            # First pads the columns
            pad_width = ((0, max(0, data.shape[0] - data.shape[0])),
                         (0, max(0, target_shape[1] - data.shape[1])))            
            data = np.pad(data, pad_width, mode='constant', constant_values=0)

            # Then pads the rows. np.pad and np.vpad did not work as expected, so using np.concatenate in this special case instead
            rows_to_pad = max(0, chunk_length_pixels - data.shape[0])
            padding_above = np.zeros((rows_to_pad, chunk_length_pixels))
            data = np.concatenate((padding_above, data), axis=0)
        
        elif bounds[3] == 80:
            # Calculate the amount of padding needed for each dimension
            pad_width = ((max(0, target_shape[0] - data.shape[0]), 0),
                         (0, max(0, target_shape[1] - data.shape[1])))
            dask_print(f"  Rows in {key} {bounds_str} in {tile_id} for {year} need padding above: {timestr()}")
            data = np.pad(data, pad_width, mode='constant', constant_values=0)
        
        elif bounds[3] == 40:
            # Calculate the amount of padding needed for each dimension
            pad_width = ((0, max(0, target_shape[0] - data.shape[0])),
                         (0, max(0, target_shape[1] - data.shape[1])))
            dask_print(f"  Rows in {key} {bounds_str} in {tile_id} for {year} need padding below: {timestr()}")
            data = np.pad(data, pad_width, mode='constant', constant_values=0)
        
        else:
            dask_print(f"  Rows in {key} {bounds_str} in {tile_id} for {year} do not need padding above or below: {timestr()}")

        # Pads the columns to 40,000 columns
        if data.shape[1] < chunk_length_pixels:
            pad_width = ((0, max(0, target_shape[0] - data.shape[0])),
                         (0, max(0, target_shape[1] - data.shape[1])))
            dask_print(f"  Columns in {key} {bounds_str} in {tile_id} for {year} do need padding to the right: {timestr()}")
            data = np.pad(data, pad_width, mode='constant', constant_values=0)
        
        else:
            dask_print(f"  Columns in {key} {bounds_str} in {tile_id} for {year} do not need padding to the right: {timestr()}")
        
        file_name_intermediate = f'{tile_id}__{bounds_str}__{key}__{year}__{timestr()}_intermediate'
        # file_name_final = f'{tile_id}__{bounds_str}__{key}__{year}__{timestr()}'  # for testing
        file_name_final = f'{tile_id}_{key}__{year}'
        upload_file = f"{value[1]}{file_name_final}.tif"

        dask_print(f"Saving {key} {bounds_str} in {tile_id} for {year} locally: {timestr()}")

        # This doesn't actually successfully create rasters with 400x400 windows. They were instead chunk_length_pixels x 400. 
        # Thus, I'm using rasterio to create an intermediate raster with partially correct dimensions, then correcting it with gdal_translate
        with rasterio.open(f"/tmp/{file_name_intermediate}.tif", 'w', driver='GTiff', width=chunk_length_pixels, height=chunk_length_pixels, count=1, dtype='uint8', 
                           crs='EPSG:4326', transform=transform, compress='lzw', blockxsize=block_size, blockysize=block_size) as dst:
            dst.write(data.astype(rasterio.uint8), 1)

        input_path = f"/tmp/{file_name_intermediate}.tif"
        output_path = f"/tmp/{file_name_final}.tif"
        
        input_dataset = gdal.Open(input_path)
        
        # Get information from the intermediate dataset
        width = input_dataset.RasterXSize
        height = input_dataset.RasterYSize
        count = input_dataset.RasterCount
        dtype = input_dataset.GetRasterBand(1).DataType
        crs = input_dataset.GetProjection()
        
        # Set final output creation options, including block size
        options = [
            'TILED=YES',
            f'BLOCKXSIZE={block_size}',
            f'BLOCKYSIZE={block_size}',
            'COMPRESS=LZW',
        ]
        
        # Create the output dataset using gdal.Translate
        gdal.Translate(
            output_path,
            input_path,
            width=width,
            height=height,
            format='GTiff',
            outputType=dtype,
            creationOptions=options,
        )

        dask_print(f"Uploading {key} {bounds_str} in {tile_id} for {year} to s3: {timestr()}")
        s3_client.upload_file(f"/tmp/{file_name_final}.tif", "gfw2-data", Key=upload_file)

        dask_print(f"Cleaning up {key} {bounds_str} in {tile_id} for {year}: {timestr()}")
        
        # Close the rasterio objects
        input_dataset = None
        output_dataset = None

        # Delete the intermediate raster
        os.remove(f"/tmp/{file_name_intermediate}.tif")

        # Delete the numpy object
        del data


    # TODO: add last year of loss creation
    # tree_removal_latest_date_uri = f'{general_uri}202312_published/tree_removal_latest_date/raw/Tree_removal_latest_date.tif'
    # futures[executor.submit(get_tile_dataset_rio, tree_removal_latest_date_uri, bounds, chunk_length_pixels)] = f"tree_removal_latest_date"

In [80]:
%%time

# Year to start the analysis
start_year = 2001   # all years
# start_year = 2012  # last few years
# start_year = 2020  # last two years
# start_year = 2021  # final year

years = list(range(start_year, 2022))
print(f"Years to iterate through: {years}")

# Area to analyze
# chunk_params arguments: W, S, E, N, chunk size (degrees)
chunk_params = [-10, 30, 40, 80, 10]  # all of Europe (25 chunks)
# chunk_params = [10, 40, 20, 50, 10]    # 10x10 deg (50N_010E, doesn't require any padding), 1 chunk
# chunk_params = [10, 30, 20, 40, 10]    # 10x10 deg (40N_010E, requires row padding at bottom), 1 chunk
# chunk_params = [20, 70, 30, 80, 10]    # 10x10 deg (80N_020E, requires row padding at top), 1 chunk
# chunk_params = [30, 70, 40, 80, 10]    # 10x10 deg (80N_030E, requires row padding at top and right (NE corner tile)), 1 chunk
# chunk_params = [30, 60, 40, 70, 10]    # 10x10 deg (70N_030E, requires column padding), 1 chunk

# Makes list of chunks to analyze
chunks = get_chunk_bounds(chunk_params)  
print(f"Processing {len(chunks)*len(years)} tasks: {len(chunks)} chunks by {len(years)} years")

# Creates list of tasks to run (1 task = 1 chunk * 1 year combo).
# I made each task a chunk x year combo because the years don't need to be done sequentially and it was taking too much memory
# to download all the inputs upfront.
# Since I wasn't downloading all the inputs upfront (instead downloading each year of data as I needed it), 
# I figured I might as well make the tasks into chunk x year combinations. 
delayed = [dask.delayed(translate_to_GLAD_tile)(chunk, chunk_params[4], year) for chunk in chunks for year in years]

# Actually runs analysis
results = dask.compute(*delayed)

Years to iterate through: [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
Processing 525 tasks: 25 chunks by 21 years
Data in chunk 10_70_20_80 for 2019. Proceeding: 20231227_16_41_20.
  Rows in tree_height 10_70_20_80 in 80N_010E for 2019 need padding above: 20231227_16_41_21
  Columns in tree_height 10_70_20_80 in 80N_010E for 2019 do not need padding to the right: 20231227_16_41_21
Saving tree_height 10_70_20_80 in 80N_010E for 2019 locally: 20231227_16_41_21
No data in chunk 0_70_10_80 for 2006. Skipping: 20231227_16_41_22
Data in chunk 10_70_20_80 for 2011. Proceeding: 20231227_16_41_23.
No data in chunk 0_70_10_80 for 2011. Skipping: 20231227_16_41_23
  Rows in tree_height 10_70_20_80 in 80N_010E for 2011 need padding above: 20231227_16_41_23
  Columns in tree_height 10_70_20_80 in 80N_010E for 2011 do not need padding to the right: 20231227_16_41_24
Saving tree_height 10_70_20_80 in 80N_010E for 2011 

In [ ]:
# Cuts Europe-size rasters to GLAD 10x10 tiles
# This does not work. It transfers data in the 80N tile to the top of the new tile, which is incorrect
def warp_to_GLAD_tile(bounds, chunk_length_deg, year):
 
    futures = {}
    layers = {}

    bounds_str = "_".join([str(round(x)) for x in bounds])
    chunk_length_pixels = int(chunk_length_deg * (40000/10))

    block_size = 400

    # Submit requests to S3 for input chunks but don't actually download them yet. This queueing of the requests before downloading them speeds up the downloading
    with concurrent.futures.ThreadPoolExecutor() as executor:
        
        tile_id = xy_to_tile_id(bounds[0], bounds[3])

        tree_height_uri = f'{input_uri}tree_height/raw/Tree_height_{year}.tif'
        tree_extent_uri = f'{input_uri}tree_extent/raw/Tree_extent_{year}.tif'
        tree_removal_uri = f'{input_uri}tree_removal/raw/Tree_removal_{year}.tif'
        
        futures[executor.submit(get_tile_dataset_rio, tree_height_uri, bounds, chunk_length_pixels)] = f"tree_height_{year}"
        # futures[executor.submit(get_tile_dataset_rio, tree_extent_uri, bounds, chunk_length_pixels)] = f"tree_extent_{year}"
        # futures[executor.submit(get_tile_dataset_rio, tree_removal_uri, bounds, chunk_length_pixels)] = f"tree_removal_{year}"


    # Wait for requests to come back with data from S3
    for future in concurrent.futures.as_completed(futures):
        layer = futures[future]
        layers[layer] = future.result()

    # Skips chunk if it has no forest extent in it
    if not np.any(layers[f"tree_height_{year}"]):
        dask_print(f"No data in chunk {bounds_str} for {year}. Skipping: {timestr()}")
        return 
    
    dask_print(f"Data in chunk {bounds_str} for {year}. Proceeding: {timestr()}.")

    transform = rasterio.transform.from_bounds(*bounds, width=chunk_length_pixels, height=chunk_length_pixels)

    # Output files to upload to s3
    output_dict = {
        "tree_height": [layers[f"tree_height_{year}"], f"forest_change/GLAD_Europe_height_data/202312_published/tree_height/{output_suffix}"],
        # "tree_extent": [layers[f"tree_extent_{year}"], f"forest_change/GLAD_Europe_height_data/202312_published/tree_extent/{output_suffix}"],
        # "tree_removal": [layers[f"tree_removal_{year}"], f"forest_change/GLAD_Europe_height_data/202312_published/tree_removal/{output_suffix}"]                          
    }

    s3_client = boto3.client("s3")

    # For every output file, saves from array to local raster, then to s3.
    # Can't save directly to s3, unfortunately, so need to save locally first.
    for key, value in output_dict.items():

        dask_print(f"Saving {key} {bounds_str} in {tile_id} for {year} locally: {timestr()}")

        input_s3_path = "/vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202312_published/tree_height/raw/Tree_height_2021.tif"

        out_tile = f'{tile_id}_{key}_{year}.tif'
        output_path = f"/tmp/{out_tile}"
        block_size = 400

        cmd = ['gdalwarp', '-t_srs', 'EPSG:4326', '-co', 'COMPRESS=DEFLATE', 
               '-co', 'BLOCKXSIZE=400', '-co', 'BLOCKYSIZE=400',
               # '-co', 'TILED=YES', '-co', 'BLOCKXSIZE=400', '-co', 'BLOCKYSIZE=400',
               '-tr', str(0.00025), str(0.00025), '-tap', '-te',
            str(bounds[0]), str(bounds[1]), str(bounds[2]), str(bounds[3]), '-dstnodata', '0', '-ot', 'byte', '-overwrite', input_s3_path, output_path]

        check_call(cmd)

        dask_print(f"Uploading {key} {bounds_str} in {tile_id} for {year} to s3: {timestr()}")
        s3_client.upload_file(output_path, "gfw2-data", Key=f"{value[1]}{out_tile}")

        dask_print(f"Cleaning up {key} {bounds_str} in {tile_id} for {year}: {timestr()}")
        os.remove(output_path)
 


    # TODO: add last year of loss creation
    # tree_removal_latest_date_uri = f'{general_uri}202312_published/tree_removal_latest_date/raw/Tree_removal_latest_date.tif'
    # futures[executor.submit(get_tile_dataset_rio, tree_removal_latest_date_uri, bounds, chunk_length_pixels)] = f"tree_removal_latest_date"

In [ ]:
tile_id = '80N_020E'
bounds = [20, 70, 30, 80]
input_s3_path = "/vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202312_published/tree_height/raw/Tree_height_2021.tif"

out_tile = '80N_020E_height_2021.tif'
output_path = f"/tmp/{out_tile}"
block_size = 400
dask_print(bounds)

cmd = ['gdalwarp', '-t_srs', 'EPSG:4326', '-co', 'COMPRESS=DEFLATE', '-tr', str(0.00025), str(0.00025), '-tap', '-te',
    str(bounds[0]), str(bounds[1]), str(bounds[2]), str(bounds[3]), '-dstnodata', '0', '-ot', 'byte', '-overwrite', input_s3_path, output_path]

check_call(cmd)

In [ ]:
coiled_client.restart() 

In [ ]:
# To run without dask at all
process_chunk([10, 49, 11, 50], 1, start_year)

In [ ]:
# To make 10x10 tiles:
# gdalwarp from subprocess.check_call(cmd) isn't working
# cmd = ['gdalwarp', '-tr', '0.00025', '0.00025', '-co', 'COMPRESS=DEFLATE', '-tap', '-te', str(10), str(49), str(11), str(50), '-dstnodata', '0', '-t_srs', 'EPSG:4326', 
#        '-overwrite', '-progress', '/vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/FH_2021.tif', 'C:\\GIS\\Carbon_model_Europe\\outputs\\50N_010E_FH_2021.tif']
# check_call(cmd)
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/FH_2021.tif 50N_010E_FH_2021.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/FH_2020.tif 50N_010E_FH_2020.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/FH_2019.tif 50N_010E_FH_2019.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/FH_2018.tif 50N_010E_1FH_2018.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/DFL_2021.tif 50N_010E_DFL_2021.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/DFL_2020.tif 50N_010E_DFL_2020.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/DFL_2019.tif 50N_010E_DFL_2019.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/DFL_2018.tif 50N_010E_DFL_2018.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_drivers/processed/drivers_2022/20230407/50N_010E_tree_cover_loss_driver_processed.tif 50N_010E_1deg_tree_cover_loss_driver_processed.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_fires/20230315/processed/50N_010E_tree_cover_loss_fire_processed.tif 50N_010E_1deg_tree_cover_loss_fire_processed.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/climate/carbon_model/other_emissions_inputs/peatlands/processed/20230315/50N_010E_peat_mask_processed.tif 50N_010E_1deg_peat_mask_processed.tif
